In [42]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r"C:\Users\Prince\Downloads\Train_Bot.json").read()
intents = json.loads(data_file)

In [43]:
# preprocessing the json data
# tokenization
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Prince\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
# lemmatize, lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))

134 documents
9 classes ['Bot', 'Exit', 'Intro', 'Joke', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket']
166 unique lemmatized words ['a', 'able', 'access', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'am', 'an', 'ann', 'anyone', 'are', 'artificial', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bored', 'bot', 'buddy', 'classification', 'contact', 'create', 'cross', 'cya', 'day', 'deep', 'did', 'diffult', 'do', 'ensemble', 'epoch', 'explain', 'feeling', 'first', 'for', 'forest', 'forward', 'from', 'function', 'funny', 'good', 'goodbye', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hour', 'how', 'hyper', 'i', 'imputer', 'in', 'intelligence', 'is', 'jerk', 'joke', 'knn', 'later', 'laugh', 'layer', 'learner', 'learning', 'leaving', 'link', 'listen', 'logistic', 'lot', 'machine', 'make', 'me', 'ml', 'my', 'naive', 'name', 'nb', 'net', 'network', 'neural', 'no', 'not', 'of', 'olympus', 'oly

>>> import nltk
>>> nltk.download()

In [45]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


In [46]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=400, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/400
27/27 [==============================] - 1s 1ms/step - loss: 2.1646 - accuracy: 0.1567 
Epoch 2/400
27/27 [==============================] - 0s 1ms/step - loss: 2.0536 - accuracy: 0.2463
Epoch 3/400
27/27 [==============================] - 0s 2ms/step - loss: 1.9328 - accuracy: 0.2612
Epoch 4/400
27/27 [==============================] - 0s 1ms/step - loss: 1.8597 - accuracy: 0.2910
Epoch 5/400
27/27 [==============================] - 0s 2ms/step - loss: 1.7361 - accuracy: 0.3955
Epoch 6/400
27/27 [==============================] - 0s 1ms/step - loss: 1.6520 - accuracy: 0.4104
Epoch 7/400
27/27 [==============================] - 0s 1ms/step - loss: 1.5637 - accuracy: 0.4179
Epoch 8/400
27/27 [==============================] - 0s 1ms/step - loss: 1.3891 - accuracy: 0.5299
Epoch 9/400
27/27 [==============================] - 0s 2ms/step - loss: 1.3127 - accuracy: 0.5821
Epoch 10/400
27/27 [==============================] - 0s 2ms/step - loss: 1.2097 - accuracy: 0.5448
Epoch 11

In [47]:
# load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open(r"C:\Users\Prince\Downloads\Train_Bot.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [60]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [68]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [69]:

def predict_class(sentence, model):
     
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [70]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [71]:
# function to start the chat bot which will continue till the user type 'end'

def start_chat():
    print("Bot: This is FOREBot! Your Personal Assistant.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [73]:
# start the chat bot
start_chat()

Bot: This is FOREBot! Your Personal Assistant.


Hello
1/1 [==============================] - 0s 16ms/step
Bot: Hello! how can i help you ?

--------------------------------------------------
Hi
1/1 [==============================] - 0s 16ms/step
Bot: Hello! how can i help you ?

--------------------------------------------------
What is your name
1/1 [==============================] - 0s 16ms/step
Bot: My name is FOREBot

--------------------------------------------------
I am bored
1/1 [==============================] - 0s 16ms/step
Bot: What is the difference between the bird flu and the swine flu? One requires tweetment and the other an oinkment.

--------------------------------------------------
What is supervised learning
1/1 [==============================] - 0s 15ms/step
Bot: This might help: https://www.geeksforgeeks.org/machine-learning/ 

--------------------------------------------------
What are epochs
1/1 [==============================] - 0s 14ms/step
Bot: This might he

##### This Code was curated with the help of the following source - https://github.com/vishal-verma27/Building-a-Simple-Chatbot-in-Python-using-NLTK 

###### We Changed the JSON file according to our needs and renamed the bot as FOREBot as well as reduced the loss of the NN model

###### We built this as our project as we were intrigued by how the chatbots function and specially during the automation era we believed that this already has immense scope and we should know the working mechanism. Specially after learning bot deep learning and natural language processing we believe that this project gives us a better understanding of the collaboration among the two.